In [1]:
import pandas as pd
from newspaper import Article
from tqdm import tqdm 
import nltk
import networkx as nx
import matplotlib.pyplot as plt
from jaal import Jaal

/Users/behemoth/anaconda3/lib/python3.11/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


## SEO reccomendation and trending topics over time 

We scraped the articles using beautiful soup to create a SEO reccomender

In [17]:
#we have gathered some basic info let us scrape the data from the article now 
import requests
from bs4 import BeautifulSoup
import time
import random

def scrape_article_data(df):
    scraped_articles = []
    for article_url in df['page_url']:
        time.sleep(random.uniform(0.5, 5.0))  
        headers = {'User-Agent': 'Mozilla/5.0'}  
        response = requests.get(article_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            title = soup.find('h1').text.strip() if soup.find('h1') else 'Unknown'
            article_text = ' '.join([p.text for p in soup.find_all('p')])
            authors = soup.find('span', class_='author-name').text.strip() if soup.find('span', class_='author-name') else 'Unknown'
            publish_date = soup.find('time').text.strip() if soup.find('time') else 'Unknown'
            scraped_data = {
                'url': article_url,
                'title': title,
                'text': article_text,
                'authors': authors,
                'publish_date': publish_date
            }
            scraped_articles.append(scraped_data)
        else:
            print(f"Failed to retrieve the article from {article_url}. Status code: {response.status_code}")
    return scraped_articles

#scraped articles for top 15 news sources 

'''

courierdic = scrape_article_data(df_courier)


preesanddic = scrape_article_data(df_pressandjournal)
rspvdic = scrape_article_data(df_rsvplive)
hufftingdic = scrape_article_data(df_huffington)
licolndic = scrape_article_data(df_lincolnshire)
df_stylist_dictionary = scrape_article_data(df_stylist)
df_ok_dictionary = scrape_article_data(df_ok)
df_corkbeo_dictionary = scrape_article_data(df_corkbeo)
Cityam_dictionary = scrape_article_data(df_cityam)
Dublin_dictionary = scrape_article_data(df_dublinlive)
Hertfordshire_dictionary = scrape_article_data(df_hertfordshire)
Empire_dictionary = scrape_article_data(df_empireonline)
Sussex_dictionary = scrape_article_data(df_sussexlive)
Hampshire_dictionary = scrape_article_data(df_hampshirelive)
and converted them to json

with open('ok.json', 'w') as file:
    json.dump(df_ok_dictionary, file)
'''


"\n\ncourierdic = scrape_article_data(df_courier)\n\n\npreesanddic = scrape_article_data(df_pressandjournal)\nrspvdic = scrape_article_data(df_rsvplive)\nhufftingdic = scrape_article_data(df_huffington)\nlicolndic = scrape_article_data(df_lincolnshire)\ndf_stylist_dictionary = scrape_article_data(df_stylist)\ndf_ok_dictionary = scrape_article_data(df_ok)\ndf_corkbeo_dictionary = scrape_article_data(df_corkbeo)\nCityam_dictionary = scrape_article_data(df_cityam)\nDublin_dictionary = scrape_article_data(df_dublinlive)\nHertfordshire_dictionary = scrape_article_data(df_hertfordshire)\nEmpire_dictionary = scrape_article_data(df_empireonline)\nSussex_dictionary = scrape_article_data(df_sussexlive)\nHampshire_dictionary = scrape_article_data(df_hampshirelive)\nand converted them to json\n\nwith open('ok.json', 'w') as file:\n    json.dump(df_ok_dictionary, file)\n"

## New df with sentiment analysis included with VADER
Created a bigger dataframe from the single dictionaries and added a newsource column

In [18]:
'''
import os
import json

json_files = [file for file in os.listdir('.') if file.endswith('.json')]

dfs = []
for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
        df_to_append = pd.DataFrame(data)
        dfs.append(df_to_append)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df.rename(columns={'url': 'page_url'}, inplace=True)



# Join the merged dataframe with the original dataframe based on the 'url' column
scraped_df = pd.merge(df, merged_df, on='page_url', how='left')
scraped_df
'''

,id,label,wikiLink,score,wikidataId,page_url,clean_url,processed_url,news_source,page_type,title,text,authors,publish_date
0,7,Rail transport,http://en.wikipedia.org/Category:Rail_transport,1.00000,Q3565868,https://www.dublinlive.ie/news/dublin-traffic-...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie,news,"Dublin traffic LIVE: Crash near Lucan, Irish R...",We will bring you all the latest traffic updat...,Unknown,
1,15,Sustainable transport,http://en.wikipedia.org/Category:Sustainable_t...,0.92090,Q1364310,https://www.dublinlive.ie/news/dublin-traffic-...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie,news,"Dublin traffic LIVE: Crash near Lucan, Irish R...",We will bring you all the latest traffic updat...,Unknown,
2,21,M7 motorway (Ireland),http://en.wikipedia.org/M7_motorway_(Ireland),0.65780,Q3775884,https://www.dublinlive.ie/news/dublin-traffic-...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie,news,"Dublin traffic LIVE: Crash near Lucan, Irish R...",We will bring you all the latest traffic updat...,Unknown,
3,28,Vehicles,http://en.wikipedia.org/Category:Vehicles,0.56240,Q42889,https://www.dublinlive.ie/news/dublin-traffic-...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie,news,"Dublin traffic LIVE: Crash near Lucan, Irish R...",We will bring you all the latest traffic updat...,Unknown,
4,38,Passenger rail transport,http://en.wikipedia.org/Category:Passenger_rai...,0.50640,Q1433891,https://www.dublinlive.ie/news/dublin-traffic-...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie/news/dublin-traffic-live-crash-n...,dublinlive.ie,news,"Dublin traffic LIVE: Crash near Lucan, Irish R...",We will bring you all the latest traffic updat...,Unknown,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271912,100,Cancer,http://en.wikipedia.org/Category:Cancer,0.16090,Q12078,https://www.cheshire-live.co.uk/news/chester-c...,cheshire-live.co.uk/news/chester-cheshire-news...,cheshire-live.co.uk/news/chester-cheshire-news...,cheshire-live.co.uk,news,NaN,NaN,NaN,NaN
1271913,110,Coronavirus-associated diseases,http://en.wikipedia.org/Category:Coronavirus-a...,0.13780,Q107361528,https://www.cheshire-live.co.uk/news/chester-c...,cheshire-live.co.uk/news/chester-cheshire-news...,cheshire-live.co.uk/news/chester-cheshire-news...,cheshire-live.co.uk,news,NaN,NaN,NaN,NaN
1271914,116,Fascia,http://en.wikipedia.org/Category:Fascia,0.12180,Q936531,https://www.cheshire-live.co.uk/news/chester-c...,cheshire-live.co.uk/news/chester-cheshire-news...,cheshire-live.co.uk/news/chester-cheshire-news...,cheshire-live.co.uk,news,NaN,NaN,NaN,NaN
1271915,136,Coronaviruses,http://en.wikipedia.org/Category:Coronaviruses,0.09304,Q290805,https://www.cheshire-live.co.uk/news/chester-c...,cheshire-live.co.uk/news/chester-cheshire-news...,cheshire-live.co.uk/news/chester-cheshire-news...,cheshire-live.co.uk,news,NaN,NaN,NaN,NaN


In [19]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


In [20]:
# Initialize VADER
sia = SentimentIntensityAnalyzer()

# Function to get sentiment
def get_sentiment(text):
    return sia.polarity_scores(text)['compound']  # Returns the compound score

# Apply the function to your DataFrame
merged_df['sentiment'] = merged_df['text'].apply(get_sentiment)

df_keywords = merged_df.drop_duplicates(subset=['page_url'])
df_keywords

,page_url,title,text,authors,publish_date,sentiment
0,https://www.corkbeo.ie/news/another-terrible-r...,Another terrible road tragedy as 3-year-old gi...,Five people have died on Irish roads in the la...,Unknown,,-0.9963
1,https://www.corkbeo.ie/news/derry-girls-siobhn...,Derry Girls' Siobhán McSweeney raises thousand...,The star announced the winner of her stunning ...,Unknown,,0.9944
2,https://www.corkbeo.ie/news/local-news/young-b...,Young boy who passed away in Cork Harbour trag...,The 14-year-old had been out swimming with fri...,Unknown,,0.8735
3,https://www.corkbeo.ie/news/local-news/four-yo...,Four young people killed in Co Tipperary car c...,A man in his early 20s and three female teenag...,Unknown,,-0.9654
4,https://www.corkbeo.ie/news/local-news/one-man...,One man arrested as Gardaí raid property in No...,Gardaí seized a huge haul of suspected cannabi...,Unknown,,-0.6553
...,...,...,...,...,...,...
3397,https://cityam.com/new-clinical-trial-hub-to-o...,New clinical trial hub to open in Canary Wharf...,Featured Submit a story Tell us your story. Fe...,Unknown,Tuesday 29 August 2023 7:42 am,0.9528
3398,https://cityam.com/even-on-their-favourite-bat...,"Even on their favourite battlegrounds, the Con...",Featured Submit a story Tell us your story. Fe...,Unknown,Thursday 31 August 2023 5:30 am,-0.9926
3399,https://cityam.com/ftse-100-live-london-flat-o...,FTSE 100 Close: London finishes in red on fear...,Featured Submit a story Tell us your story. Fe...,Unknown,Thursday 31 August 2023 10:17 am,0.9890
3400,https://cityam.com/glencores-shares-slide-as-1...,Glencore’s shares slide as 197 claimants for l...,Featured Submit a story Tell us your story. Fe...,Unknown,Thursday 31 August 2023 11:46 am,-0.8360


In [21]:
import spacy
nlp = en_core_web_sm.load()

# Function to extract keywords using Spacy
def extract_keywords(text):
    try:
        doc = nlp(text)
        keywords = [(ent.text, ent.label_) for ent in doc.ents][:10]  # Extract the first 10 entities
    except NameError:
        return []
    return keywords

# Apply the function to extract keywords from df_keywords
df_keywords['keywords'] = df_keywords['text'].apply(lambda x: extract_keywords(x))

# Display the keywords for the first few articles
print(df_keywords[['text', 'keywords']].head())



NameError: name 'en_core_web_sm' is not defined

In [ ]:
df_keywords.to_csv('df_withkeywords.csv')
# Extract unique values from the 'keywords' column and print them as a list of lists
unique_keywords = df_keywords['keywords'].explode().unique()
unique_keyword_types = [keyword[1] for keyword in unique_keywords if isinstance(keyword, tuple)]
print(set(unique_keyword_types))


{'FAC', 'LOC', 'TIME', 'WORK_OF_ART', 'QUANTITY', 'LANGUAGE', 'EVENT', 'PERCENT', 'ORG', 'NORP', 'PRODUCT', 'PERSON', 'DATE', 'GPE', 'CARDINAL', 'LAW', 'MONEY', 'ORDINAL'}


In [ ]:
from dateutil import parser
def standardize_date(date_str):
    if pd.isnull(date_str):  # Check if date_str is null
        return None

    standardized_date = pd.NaT  # Defaulting to 'Not a Time' for error handling

    # Preprocess the date string to handle different formats
    if isinstance(date_str, str):  # Check if date_str is a string
        date_str = date_str.replace(',', '').replace('BST', '').replace('UTC', '')
        date_str = date_str.split(' ')[-3:] if 'am' in date_str.lower() or 'pm' in date_str.lower() else date_str
        date_str = ' '.join(date_str).strip() if isinstance(date_str, list) else date_str
        # Handle specific cases like '18:29, 24 Aug 2023' by reversing the order to '24 Aug 2023 18:29'
        if ':' in date_str.split(' ')[0]:
            parts = date_str.split(', ')
            if len(parts) > 1:
                time_part, date_part = parts
                date_str = f"{date_part} {time_part}"
        
        # Try parsing the date
        try:
            standardized_date = parser.parse(date_str)
        except ValueError:
            # If parsing fails, handle the specific cases
            pass

    # Return the formatted date as "Month Year" (e.g., "April 2023")
    return standardized_date.strftime('%B %Y') if pd.notnull(standardized_date) else None

from dateutil import parser

def standardize_date(date_str):
    standardized_date = pd.NaT  # Defaulting to 'Not a Time' for error handling

    # Remove any time or timezone information as we don't care about it
    date_str = date_str.split(',')[0].split('BST')[0].strip()
    
    # Try parsing the date
    try:
        standardized_date = parser.parse(date_str)
    except ValueError:
        # If parsing fails, handle the specific cases
        pass

    return standardized_date.strftime('%B %Y') if pd.notnull(standardized_date) else None




In [ ]:
df_w_dates['publish_date'] = df_w_dates['publish_date'].apply(standardize_date)


In [ ]:
df_w_dates.head()
# Export the DataFrame df_w_dates to a CSV file
df_w_dates.to_csv('df_w_dates.csv', index=False)



# Creating the Network Visualisation

## Started by creating the network based on previously processes data

In [40]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from collections import defaultdict


shared_labels = defaultdict(lambda: defaultdict(int))
for _, row in df.iterrows():
    shared_labels[row['news_source']][row['label']] += 1


total_articles = df['news_source'].value_counts()

top_three_labels_per_source = df.groupby('news_source')['label'].agg(lambda x: x.value_counts().nlargest(3).index.tolist())


G = nx.Graph()

# Add nodes with size attribute and top three labels as a title
for news_source, count in total_articles.items():
    G.add_node(news_source, size=count, title=", ".join(top_three_labels_per_source[news_source]))

# determine the connections based on shared labels and find the most common label for each connection
connections = defaultdict(list)
most_common_label_per_connection = {}
for source_a in shared_labels:
    for source_b in shared_labels:
        if source_a != source_b:
            common_labels = set(shared_labels[source_a]) & set(shared_labels[source_b])
            weight = sum(min(shared_labels[source_a][label], shared_labels[source_b][label]) for label in common_labels)
            if weight > 0:
                filtered_common_labels = [label for label in common_labels if shared_labels[source_a][label] > 0.5 and shared_labels[source_b][label] > 0.5]
                if filtered_common_labels:
                    most_common_label = max(filtered_common_labels, key=lambda label: min(shared_labels[source_a][label], shared_labels[source_b][label]))
                    connections[source_a].append((source_b, weight))
                    most_common_label_per_connection[(source_a, source_b)] = most_common_label
# Sort connections based on weight and limit to top 1-3 connections per node
for source, edges in connections.items():
    sorted_edges = sorted(edges, key=lambda x: x[1], reverse=True)[:3]  # Keep only top 1-3 connections based on weight
    for target, weight in sorted_edges:
        G.add_edge(source, target, weight=weight, label=most_common_label_per_connection[(source, target)])

# Normalize node sizes for visualization
max_size = max(nx.get_node_attributes(G, 'size').values())
min_size = 10  # Minimum node size
for node in G.nodes:
    G.nodes[node]['size'] = min_size + (G.nodes[node]['size'] / max_size) * 50  # Adjust this formula as needed



## Creating the final network visualisation in Pyvis

In [41]:
nt = Network(height="750",notebook=True, cdn_resources='remote', bgcolor="#181717", font_color="white", select_menu=True, filter_menu=True)

# Set the physics layout of the network
nt.options.physics.enabled = True



top_5_nodes = sorted(G.nodes(data=True), key=lambda x: x[1]['size'], reverse=True)[:5]
top_5_labels = [node[0] for node in top_5_nodes]
for node in G.nodes(data=True):
    if node[0] in top_5_labels:
        # For top 5 nodes, include the label outside
        title = node[1]['title'].split(',')[0] if ',' in node[1]['title'] else node[1]['title']
        nt.add_node(node[0], size=node[1]['size'], title=node[1]['title'], label =  f"{node[0]} - {title}" ,color='440154', font={'size': 20})
    else:
        title = node[1]['title'].split(',')[0] if ',' in node[1]['title'] else node[1]['title']
        nt.add_node(node[0], size=node[1]['size'], label=node[0] , title=node[1]['title'], color='89D28B')
        
for edge in G.edges(data=True):
    source = edge[0]
    target = edge[1]
    weight = edge[2]['weight']
    nt.add_edge(source, target, value=weight * 0.5, color='8EA8F2', title=edge[2]['label'])
nt.title = "News Source Network Visualization"


nt.set_options('''{
  "nodes": {
    "borderWidth": null,
    "borderWidthSelected": null,
    "opacity": 0.9,
      "scaling": {
      "label": {
        "enabled": true,
        "min": 21,
        "max": 30,
        "maxVisible": 35,
        "drawThreshold": 1
      }
      },
 
    "size": null
  },
  "edges": {
    "color": {
      "inherit": true
    }
  },
  "interaction": {
    "hover": true
  }
}''')


nt.show("network_visualization.html")

network_visualization.html
